# CHEMBL Filtering and Clustering

In [1]:
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors, PandasTools, Crippen
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.inchi import MolToInchiKey
from rdkit import RDLogger
from tqdm.auto import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import csv

# Silence RDKit noise
RDLogger.DisableLog("rdApp.*")

%matplotlib inline


Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


## Helper Functions

In [ ]:
# RDKit standardization helpers
lfc = rdMolStandardize.LargestFragmentChooser()
normalizer = rdMolStandardize.Normalizer()
uncharger = rdMolStandardize.Uncharger()

def standardize_and_get_inchikey(mol, strip_stereo=True):
    """
    Standardize a molecule, optionally strip stereochemistry, and get:
      - standardized Mol
      - InChIKey (dedup key)
      - canonical SMILES (for convenience, not for dedup)

    Returns (mol_clean, inchikey, smiles) or (None, None, None).
    """
    if mol is None:
        return None, None, None

    try:
        # Largest fragment (remove salts/solvents, keep main species)
        mol = lfc.choose(mol)

        # Cleanup + normalize + uncharge
        mol = rdMolStandardize.Cleanup(mol)
        mol = normalizer.normalize(mol)
        mol = uncharger.uncharge(mol)

        # Sanitize structure
        Chem.SanitizeMol(mol)

        # Optionally collapse stereochemistry to avoid stereo-duplicated entries
        if strip_stereo:
            Chem.RemoveStereochemistry(mol)

        # InChIKey **after** standardization 
        ik = MolToInchiKey(mol)

        # Canonical SMILES 
        smiles = Chem.MolToSmiles(mol, canonical=True, isomericSmiles=not strip_stereo)

        # Rebuild from SMILES to have a clean canonical representation
        mol_clean = Chem.MolFromSmiles(smiles)
        if mol_clean is None:
            return None, None, None
        Chem.SanitizeMol(mol_clean)

        return mol_clean, ik, smiles

    except Exception:
        return None, None, None

### 1.Deduplication

In [ ]:
chembl_sdf_path='chembl_36.sdf'
dedup_sdf_path='chembl_dedup.sdf'
n_bins=60

supplier = Chem.SDMolSupplier(chembl_sdf_path, sanitize=False, removeHs=True)
writer = Chem.SDWriter(dedup_sdf_path)

seen_inchikeys = set()
rows = []

total = 0
invalid = 0
duplicates = 0
written = 0

for mol in tqdm(supplier, desc="Cleaning and deduplicating (InChIKey)"):
    total += 1

    mol_clean, ik, smi = standardize_and_get_inchikey(mol, strip_stereo=True)
    if mol_clean is None or ik is None:
        invalid += 1
        continue

    if ik in seen_inchikeys:
        duplicates += 1
        continue

    seen_inchikeys.add(ik)
    writer.write(mol_clean)
    written += 1

    mw = Descriptors.MolWt(mol_clean)
    hacc = rdMolDescriptors.CalcNumHBA(mol_clean)
    rows.append((smi, ik, mw, hacc))

writer.close()

print("=== Summary ===")
print(f"Total molecules read        : {total}")
print(f"Invalid / failed molecules  : {invalid}")
print(f"Duplicate InChIKeys         : {duplicates}")
print(f"Unique molecules written    : {written}")
print(f"Distinct InChIKeys stored   : {len(seen_inchikeys)}")

### 2. Property Filtering

#### 2.1 Property Filtering Plots

In [ ]:
sdf_in_path = "chembl_dedup.sdf"
csv_out_path = "chembl_dedup_props.csv"

supplier = Chem.SDMolSupplier(sdf_in_path, sanitize=True, removeHs=True)

with open(csv_out_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "SMILES",
        "MW",
        "HeavyAtomCount",
        "RingCount",
        "LogP",
        "HBD",
        "HBA",
        "RotatableBonds"
    ])

    total = 0
    invalid = 0

    for mol in tqdm(supplier, desc="Extracting MW, HA, Rings, LogP, HBD, HBA, RB"):
        total += 1

        if mol is None:
            invalid += 1
            continue

        try:
            smi = Chem.MolToSmiles(mol, canonical=True)
            mw = Descriptors.MolWt(mol)
            ha = mol.GetNumHeavyAtoms()
            rings = rdMolDescriptors.CalcNumRings(mol)

            logp = Crippen.MolLogP(mol)
            hbd = rdMolDescriptors.CalcNumHBD(mol)
            hba = rdMolDescriptors.CalcNumHBA(mol)
            rb = rdMolDescriptors.CalcNumRotatableBonds(mol)

        except Exception:
            invalid += 1
            continue

        writer.writerow([smi, mw, ha, rings, logp, hbd, hba, rb])

print("Done.")
print("Invalid molecules:", invalid)


In [ ]:
# Columns we want to extract
usecols = [
    "MW",
    "HeavyAtomCount",
    "RingCount",
    "LogP",
    "HBD",
    "HBA",
    "RotatableBonds"
]

df = pd.read_csv(csv_out_path, usecols=usecols)

# === Compute min/max for each property ===
mw_min, mw_max = df["MW"].min(), df["MW"].max()
ha_min, ha_max = df["HeavyAtomCount"].min(), df["HeavyAtomCount"].max()
r_min,  r_max  = df["RingCount"].min(), df["RingCount"].max()
logp_min, logp_max = df["LogP"].min(), df["LogP"].max()
hbd_min, hbd_max = df["HBD"].min(), df["HBD"].max()
hba_min, hba_max = df["HBA"].min(), df["HBA"].max()
rb_min, rb_max = df["RotatableBonds"].min(), df["RotatableBonds"].max()

print (mw_min, mw_max,
 ha_min, ha_max,
 r_min, r_max,
 logp_min, logp_max,
 hbd_min, hbd_max,
 hba_min, hba_max,
 rb_min, rb_max)


In [ ]:
# === Proposed thresholds ===
MIN_MW,  MAX_MW  = 200, 700
MIN_HA,  MAX_HA  = 5, 60
MIN_RINGS, MAX_RINGS = 0, 6

MIN_LOGP, MAX_LOGP = -2, 7
MAX_HBD = 7
MAX_HBA = 12
MAX_RB  = 20

# Helper function for uniform plotting
def plot_property(df, col, bins, min_val=None, max_val=None):
    plt.figure(figsize=(6,4))
    plt.hist(df[col], bins=bins, color="steelblue", edgecolor="black")
    
    if min_val is not None:
        plt.axvline(min_val, color="red", linestyle="--")
    if max_val is not None:
        plt.axvline(max_val, color="red", linestyle="--")
    
    plt.title(f"{col} distribution")
    plt.xlabel(col)
    plt.ylabel("Count")
    plt.show()

# === MW ===
plot_property(df, "MW", bins=80, min_val=MIN_MW, max_val=MAX_MW)

# === Heavy Atom Count ===
plot_property(df, "HeavyAtomCount", bins=60, min_val=MIN_HA, max_val=MAX_HA)

# === Ring Count ===
plot_property(df, "RingCount", bins=20, min_val=MIN_RINGS, max_val=MAX_RINGS)

# === LogP ===
plot_property(df, "LogP", bins=60, min_val=MIN_LOGP, max_val=MAX_LOGP)

# === HBD ===
plot_property(df, "HBD", bins=15, max_val=MAX_HBD)

# === HBA ===
plot_property(df, "HBA", bins=20, max_val=MAX_HBA)

# === Rotatable Bonds ===
plot_property(df, "RotatableBonds", bins=40, max_val=MAX_RB)


#### 2.2 Filtering Step

In [ ]:
sdf_in_path = "chembl_dedup.sdf"                 # current deduped set
sdf_out_path = "chembl_filtered_2.sdf"             # filtered SDF
csv_out_path = "chembl_filtered_props_2.csv"       # filtered props

# Thresholds
MIN_MW, MAX_MW = 200, 700
MIN_HEAVY, MAX_HEAVY = 5, 60
MIN_RINGS, MAX_RINGS = 0, 6
MIN_LOGP = -2
MAX_LOGP = 7
MAX_HBD = 7
MAX_HBA = 12
MAX_RB  = 20

supplier = Chem.SDMolSupplier(sdf_in_path, sanitize=True, removeHs=True)
writer = Chem.SDWriter(sdf_out_path)

total = 0
invalid = 0
kept = 0
filtered_out = 0

with open(csv_out_path, "w", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow([
        "SMILES", "MW", "HeavyAtomCount", "RingCount",
        "LogP", "HBD", "HBA", "RotatableBonds"
    ])

    for mol in tqdm(supplier, desc="Filtering ChEMBL"):
        total += 1

        if mol is None:
            invalid += 1
            continue

        try:
            smi = Chem.MolToSmiles(mol, canonical=True)

            mw = Descriptors.MolWt(mol)
            heavy = mol.GetNumHeavyAtoms()
            rings = rdMolDescriptors.CalcNumRings(mol)

            logp = Crippen.MolLogP(mol)
            hbd = rdMolDescriptors.CalcNumHBD(mol)
            hba = rdMolDescriptors.CalcNumHBA(mol)
            rb = rdMolDescriptors.CalcNumRotatableBonds(mol)

        except Exception:
            invalid += 1
            continue

        # --- Apply filters ---

        if not (MIN_MW <= mw <= MAX_MW):
            filtered_out += 1
            continue

        if not (MIN_HEAVY <= heavy <= MAX_HEAVY):
            filtered_out += 1
            continue

        if not (MIN_RINGS <= rings <= MAX_RINGS):
            filtered_out += 1
            continue

        if not (MIN_LOGP <= logp <= MAX_LOGP):
            filtered_out += 1
            continue

        if hbd > MAX_HBD:
            filtered_out += 1
            continue

        if hba > MAX_HBA:
            filtered_out += 1
            continue

        if rb > MAX_RB:
            filtered_out += 1
            continue

        # --- Passed all filters → keep ---
        writer.write(mol)
        csv_writer.writerow([smi, mw, heavy, rings, logp, hbd, hba, rb])
        kept += 1

writer.close()

print("=== Filtering summary ===")
print(f"Total molecules read   : {total}")
print(f"Invalid molecules      : {invalid}")
print(f"Filtered out by rules  : {filtered_out}")
print(f"Kept (written to SDF)  : {kept}")
print(f"Output SDF             : {sdf_out_path}")
print(f"Output CSV             : {csv_out_path}")


## 3. Clustering

In [7]:
import bblean

from rdkit import Chem


THRESHOLD = 0.5       
BRANCHING = 128            # adjust to memory
NBITS = 2048              # ECFP4

In [2]:
smiles_list = []

supplier = Chem.SDMolSupplier("chembl_filtered_2.sdf", sanitize=True, removeHs=True)
for mol in supplier:
    if mol is None:
        continue
    smiles_list.append(Chem.MolToSmiles(mol))

[15:35:05] Explicit valence for atom # 10 Si, 6, is greater than permitted
[15:35:05] ERROR: Could not sanitize molecule ending on line 60766518
[15:35:05] ERROR: Explicit valence for atom # 10 Si, 6, is greater than permitted
[15:35:28] Explicit valence for atom # 7 P, 7, is greater than permitted
[15:35:28] ERROR: Could not sanitize molecule ending on line 67713996
[15:35:28] ERROR: Explicit valence for atom # 7 P, 7, is greater than permitted
[15:36:45] Explicit valence for atom # 3 Ar, 1, is greater than permitted
[15:36:45] ERROR: Could not sanitize molecule ending on line 89932534
[15:36:45] ERROR: Explicit valence for atom # 3 Ar, 1, is greater than permitted


In [8]:
fps = bblean.fps_from_smiles(
    smiles_list,
    kind="ecfp4",
    pack=True,       
    n_features=2048
)

bb = bblean.BitBirch(
    threshold=THRESHOLD,
    branching_factor=BRANCHING,
    merge_criterion="diameter"
)

bb.fit(fps)
clusters = bb.get_cluster_mol_ids()

In [9]:
import numpy as np

# 3. Convert to label vector
labels = np.zeros(len(fps), dtype=np.int32)
for cid, members in enumerate(clusters):
    for idx in members:
        labels[idx] = cid

labels  # now each molecule has a cluster ID

import pandas as pd

df = pd.DataFrame({
    "SMILES": smiles_list,
    "cluster": labels
})

df.to_csv("chembl_bitbirch_clusters_3.csv", index=False)

representatives = {}
for idx, c in enumerate(labels):
    if c not in representatives:
        representatives[c] = idx

rep_indices = np.array(list(representatives.values()))
rep_smiles = [smiles_list[i] for i in rep_indices]

df_rep = pd.DataFrame({"SMILES": rep_smiles})
df_rep.to_csv("chembl_bitbirch_representatives_3.csv", index=False)